In [1]:
import re
import pandas as pd
from tqdm import tqdm

In [4]:
def parseLog(log_file_path):
    # Regular expression to match the pattern "IP: <number> Addr: <number>"
    pattern = r"IP:\s*(\d+)\s*Addr:\s*(\d+)"

    # Lists to store extracted IP and Addr values
    ip_list = []
    addr_list = []

    # Read and process the log file
    with open(log_file_path, 'r') as file:
        for line in file:
            match = re.search(pattern, line)
            if match:
                # Append the matched IP and Addr values to the lists
                ip_list.append(int(match.group(1)))
                addr_list.append(int(match.group(2)))
    return ip_list, addr_list

In [5]:
# Define the file path for the log file
log_file_path = '../471.omnetpp-188B-ipcp_isbv2-case5-canary-ip.log'

ip_list, addr_list = parseLog(log_file_path)
# Create a DataFrame from the lists
df5 = pd.DataFrame({'IP': ip_list, 'Addr': addr_list})

# Define the file path for the log file
log_file_path = '../471.omnetpp-188B-ipcp_isbv2-case6-canary-ip.log'

ip_list, addr_list = parseLog(log_file_path)

df6 = pd.DataFrame({'IP': ip_list, 'Addr': addr_list})

In [6]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395147 entries, 0 to 395146
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   IP      395147 non-null  int64
 1   Addr    395147 non-null  int64
dtypes: int64(2)
memory usage: 6.0 MB


In [7]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601778 entries, 0 to 601777
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   IP      601778 non-null  int64
 1   Addr    601778 non-null  int64
dtypes: int64(2)
memory usage: 9.2 MB


In [8]:
df6['IP'].nunique()

250

In [9]:
df5['IP'].nunique()

174

In [10]:
df5['IP'].value_counts()/df5['IP'].value_counts().sum()

IP
4539226    0.203423
4539217    0.184678
4539246    0.098963
4539250    0.098495
4539209    0.074124
             ...   
5250301    0.000003
5251409    0.000003
5252590    0.000003
4252164    0.000003
4251300    0.000003
Name: count, Length: 174, dtype: float64

In [11]:
df6['IP'].value_counts()/df6['IP'].value_counts().sum()

IP
4539226    0.130643
4539217    0.116820
4539250    0.068110
4539246    0.065129
4539209    0.051893
             ...   
4353314    0.000002
4244348    0.000002
4233368    0.000002
5253105    0.000002
5253151    0.000002
Name: count, Length: 250, dtype: float64

In [12]:
series1 = df6['IP'].unique()
series2 = df5['IP'].unique()

In [13]:
difference_set = set(series1) - set(series2)

# Convert back to a Series if needed
difference_series = pd.Series(list(difference_set)).unique()

print(difference_series)

[4350995 4253720 4543004 4542524 4542533 4462153 4359247 4359254 4504152
 4246107 4251744 4352611 4457576 4457579 4543086 4540016 4540023 4351615
 4243072 4542592 4222094 4233368 4455584 5250213 4222118 4358821 4542632
 4580529 4233393 4251313 4580535 4580538 4203711 5250752 4539075 4499657
 4206288 4498642 4499676 4348637 4552926 4597486 4539632 4251389 4554498
 4244738 4203779 4243209 4462352 4253970 4539666 4539675 4542237 4353314
 4250918 4532528 4534065 4534071 4539706 4352846 4446032 4244821 4245338
 4206430 4457324 4244348 4455805 4242816 4540290 4243366 4253610 4457392
 4457401 4241865 4252110 4554194 4457426 4499924 4539864 4359131 4457438
 4457450 4553710 5253105 4242930 4359161]


In [14]:
difference_set = set(series2) - set(series1)

# Convert back to a Series if needed
difference_series_2 = pd.Series(list(difference_set)).unique()

print(difference_series_2)

[4579072 4204773 4579080 5252586 5252590 5251409 4540406 4942456 5253404
 5250301]


In [30]:
def parseLogs(log_file_path):
    # Parsing Prefetch decision infos
    pf_pattern_a = r"Key \(PC\): (\w+) Trigger physical address: (\w+) structural address: (\w+) prefetch target: (\w+)"
    pf_pattern_b = r"Key \(PC\): (\w+) Trigger physical address: (\w+) structural address: (\w+)$"
    table_pattern = r"Key \(PC\):(\w+) Trigger:(\w+) # candidates: (\w+)"
    pf_useful_pattern = r"ISB Useful PF: IP: (\w+)\s+Addr: (\w+)"

    # Lists to store extracted values
    pf_pc_list_a = []
    pf_physical_address_list_a = []
    pf_structural_address_list_a = []
    pf_prefetch_target_list_a = []

    pf_pc_list_b = []
    pf_physical_address_list_b = []
    pf_structural_address_list_b = []

    t_pc_list = []
    t_physical_address_list = []
    t_structural_address_list = []

    pf_useful_pc_list = []
    pf_useful_address_list = []

    # Read and process the log file line by line
    with open(log_file_path, 'r') as file:
        for line in tqdm(file):
            pf_match_a = re.search(pf_pattern_a, line)
            pf_match_b = re.search(pf_pattern_b, line)
            tb_match = re.search(table_pattern, line)
            pf_useful_match = re.search(pf_useful_pattern, line)
            if pf_match_a:
                pf_pc_list_a.append(pf_match_a.group(1))
                pf_physical_address_list_a.append(pf_match_a.group(2))
                pf_structural_address_list_a.append(pf_match_a.group(3))
                pf_prefetch_target_list_a.append(pf_match_a.group(4))
            elif pf_match_b:
                pf_pc_list_b.append(pf_match_b.group(1))
                pf_physical_address_list_b.append(pf_match_b.group(2))
                pf_structural_address_list_b.append(pf_match_b.group(3))
            elif pf_useful_match:
                pf_useful_pc_list.append(pf_useful_match.group(1))
                pf_useful_address_list.append(pf_useful_match.group(2))
            elif tb_match:
                t_pc_list.append(tb_match.group(1))
                t_physical_address_list.append(tb_match.group(3))
                t_structural_address_list.append(tb_match.group(2))

    pf_df = pd.DataFrame({
        'PC': pf_pc_list_a,
        'Trigger Physical_Address': pf_physical_address_list_a,
        'Trigger Structural_Address': pf_structural_address_list_a,
        'Prefetch Target': pf_prefetch_target_list_a
    })

    nt_df = pd.DataFrame({
        'PC': pf_pc_list_b,
        'Trigger Physical_Address': pf_physical_address_list_b,
        'Trigger Structural_Address': pf_structural_address_list_b,
    })

    useful_pf_df = pd.DataFrame({
        'PC': pf_useful_pc_list,
        'Prefetch_Address': pf_useful_address_list,
    })

    table_df = pd.DataFrame({
        'PC': t_pc_list,
        'Trigger_Address': t_structural_address_list,
        'Candidates': t_physical_address_list,
    })

    return (pf_df, nt_df, useful_pf_df, table_df)

In [31]:
_, _, _, table_df_MT = parseLogs("../471.omnetpp-188B-ipcp_isbv3-all-pf-miss-train.log")

773851it [00:01, 633825.22it/s]


In [32]:
_, _, _, table_df_AT = parseLogs("../471.omnetpp-188B-ipcp_isbv3-all-pf-all-train.log")

1166979it [00:01, 611294.84it/s]


In [35]:
table_df_AT['Candidates'].value_counts().sort_index(ascending=False)

Candidates
8       528
7       222
6       500
5       964
4      1636
3      2849
2      6180
1    536033
0    617745
Name: count, dtype: int64

In [36]:
table_df_MT['Candidates'].value_counts().sort_index(ascending=False)

Candidates
8      5264
7       527
6       493
5       606
4       662
3       806
2      1120
1    467048
0    297003
Name: count, dtype: int64

In [8]:
table_df_MT['PC'].value_counts()

PC
46267c    6998
43a5de    6993
501cf1    4037
44a9d7    4006
452f40    3827
          ... 
502ad7       1
402e77       1
4547f6       1
40c427       1
40e204       1
Name: count, Length: 196, dtype: int64

In [9]:
table_df_AT['PC'].value_counts()

PC
46267c    7051
43a5de    6996
507080    4881
501cf1    4188
44a9d7    4096
          ... 
402f9c       1
4098e3       1
454308       1
4544f0       1
454840       1
Name: count, Length: 372, dtype: int64

In [ ]:
table_df_AT

,PC,Structural_Address,Physical_Address
0,46267c,400,1e6c980
1,46267c,401,1e6ca40
2,46267c,402,1dbac00
3,46267c,403,1dbad00
4,46267c,404,1dbba80
...,...,...,...
127521,507080,24cf404,3868080
127522,4543a7,24cf800,beda440
127523,45df1d,24cfc00,7f2440
127524,507080,24d0000,3867f80


In [36]:
ipcp_isb_pf_df_ms, ipcp_isb_nt_df_ms, ipcp_isb_useful_pf_df_ms, ipcp_isb_table_df_ms = parseLogs("../471.omnetpp-188B-ipcp_isbv2-miss_pf-miss_train-tu.log")

3177766it [00:06, 460373.54it/s]


In [53]:
ipcp_isb_useful_pf_df['PC'].value_counts().sum()

np.int64(559985)

In [52]:
frequency_a = ipcp_isb_useful_pf_df_ms['PC'].value_counts()
frequency_a.sum()

np.int64(724369)

In [51]:
values_to_count = ipcp_isb_useful_pf_df_ms['PC'].value_counts().index

# Filter and get the value counts for only the specified values
frequency_b = ipcp_isb_useful_pf_df['PC'].value_counts().reindex(values_to_count).fillna(0).astype(int)

frequency_b.sum()

np.int64(511692)

### Count of how many more useful prefetches per PC in miss stream config that are not in all_pf-miss_train config

In [58]:
pd.set_option('display.max_rows', None)
delta = frequency_a - frequency_b
delta[delta.values > 0].sort_values(ascending=False)
# pd.set_option('display.max_rows', 100)

PC
4578d5    20580
40e8b6    18560
457dd2    16539
4497e9    13196
43fb2a     9351
43d720     9035
447b9f     8719
454f30     8196
502c35     8039
501c59     7914
40da8f     7594
43a5de     7540
454782     7146
44a418     6746
50262d     6624
44ba08     6196
501cf1     6147
452f34     6122
502c68     6100
40e900     6088
452f40     5804
457dbd     5691
44a9d7     5034
507080     4920
40dd26     4546
43d75c     4105
44b9d5     3960
44a8c9     3958
454f1d     3939
447bc5     3930
4024b3     3573
40e860     3395
46267c     3317
40da67     3246
40e7b5     3220
40ded8     3039
425add     3021
454647     2996
40cac6     2914
4403b9     2769
454794     2740
43d725     2731
40bfa6     2723
457dc5     2608
441649     2589
452f18     2567
44b9f8     2487
40b959     2365
40c502     2170
44a4d2     1991
44036c     1978
4626c8     1974
45506a     1937
40e818     1903
441710     1886
45def7     1836
44034a     1805
40bd2a     1694
440374     1624
502adb     1600
45e4b1     1564
40bda3     1417
45467

In [59]:
pd.set_option('display.max_rows', 100)

In [64]:
ipcp_isb_pf_df[ipcp_isb_pf_df['PC'] == '4578d5']

,PC,Trigger Physical_Address,Trigger Structural_Address,Prefetch Target
54457,4578d5,4561fc0,22800,45bae00
54458,4578d5,4561fc0,22800,4613c40
54459,4578d5,4561fc0,22800,466ca40
54460,4578d5,4561fc0,22800,46c3100
54461,4578d5,4561fc0,22800,471e680
...,...,...,...,...
5562475,4578d5,44dc280,2907,44e3640
5562476,4578d5,44dc280,2907,44e7080
5562477,4578d5,44dc280,2907,44ee900
5562478,4578d5,44dc280,2907,61cb840


In [67]:
ipcp_isb_table_df[ipcp_isb_table_df['PC'] == '4578d5']

,PC,Structural_Address,Physical_Address
1636,4578d5,2800,1dbb640
1637,4578d5,2801,1dbc940
1638,4578d5,2802,1dbdc00
1639,4578d5,2803,1dc1500
1640,4578d5,2804,1dc27c0
...,...,...,...
214207,4578d5,22dc80a,6654200
214208,4578d5,22dc80b,6650840
214209,4578d5,22dc80c,664ce40
214210,4578d5,22dc80d,61c6740


In [23]:
ipcp_isb_nt_df_ms['PC'].value_counts()

PC
44b9f8    19280
43a5de    18392
452f24    16680
507080    16536
447b9f    15227
          ...  
5025a3        1
44092c        1
44a490        1
42816c        1
50297a        1
Name: count, Length: 286, dtype: int64

In [24]:
ipcp_isb_pf_df_ms['PC'].value_counts()

PC
45435a    124868
454351    119582
454372     82240
45436e     80111
507080     48707
           ...  
40abb7         2
447be1         1
426a63         1
45df60         1
502d05         1
Name: count, Length: 246, dtype: int64

In [25]:
ipcp_isb_pf_df_ms[ipcp_isb_pf_df_ms['PC'] == '45435a']

,PC,Trigger Physical_Address,Trigger Structural_Address,Prefetch Target
12,45435a,ac969c0,1007,974db40
13,45435a,ac969c0,1007,81b2d00
14,45435a,ac969c0,1007,bbec3c0
15,45435a,ac969c0,1007,802b940
16,45435a,ac969c0,1007,802d100
...,...,...,...,...
1681519,45435a,b0fc000,3811005,889a380
1681555,45435a,80a5c80,2798401,80a8c00
1681633,45435a,b0cf0c0,289d008,beca3c0
1681634,45435a,b0d4fc0,387f000,b1d0700


In [26]:
ipcp_isb_pf_df['PC'].unique()

array(['45435a', '454351', '45436e', '454372', '454349', '45438b',
       '4543a7', '45438e', '454647', '4543a0', '457dbd', '40ea06',
       '425b50', '454801', '454808', '46c458', '46c46f', '46c443',
       '457c4c', '4542ce', '4542d2', '454794', '4542e6', '45def7',
       '4542f2', '40e928', '507080', '45df19', '4578d5', '457bee',
       '43ffe3', '45def0', '43ff95', '501c59', '457dc5', '44b9f8',
       '43d75c', '43fb0a', '45df1d', '42681b', '46c47e', '46c430',
       '426916', '46c46b', '46c451', '426a20', '426a3f', '507082',
       '426cd7', '4028e5', '4542c3', '45df3c', '4542f9', '44ba08',
       '4542e9', '452f24', '505cc8', '45434e', '447b9f', '425e79',
       '406c8e', '4079df', '45e4c6', '4403de', '502629', '502740',
       '4547cb', '502744', '5026f1', '45df60', '502ad7', '426425',
       '45df08', '4403d2', '426b87', '4403ca', '50262d', '44034d',
       '4403ea', '4079e4', '425afa', '40ba31', '425b70', '455513',
       '40da8f', '40cac6', '40da67', '5025ad', '40ca5b', '40cb

### IPs for which prefetch was triggered in all_pf-miss-train config but not in miss_stream config

In [27]:
series1 = ipcp_isb_pf_df['PC'].unique()
series2 = ipcp_isb_pf_df_ms['PC'].unique()
difference_set = set(series1) - set(series2)

# Convert back to a Series if needed
difference_series = pd.Series(list(difference_set)).unique()

print(difference_series)

['44040e' '5044d4' '50291c' '45e4bd' '40df4c' '5025ee' '454599' '40e7d3'
 '455239' '502adf' '40b969' '402eca' '4282c4' '45df00' '45551d' '4543cd'
 '426e33' '5044e6' '43ff95' '4409cb' '40c406' '405480' '4544bb' '5025eb'
 '5044dd' '502c3f' '45df08' '50287a' '4b6a5b' '502151' '40c3be' '40df3f'
 '5025f4' '4b7b34' '40e8db' '426cb8' '4403ca' '426425' '45df1d' '440933'
 '447c68' '50297a' '43ffe7' '4079df' '5072aa' '452929' '40ba3d' '4028ea'
 '43ffdf' '4547f6' '457c4c' '40a5ef' '5025ea' '402dd3' '4542ce' '40c3ee'
 '501c9c' '455522' '46c9f8' '4542c0' '45291b' '501f24' '4403e4' '40bd5c'
 '502ae3' '45462a' '46c59f' '40de94' '46c573' '40cadf' '46c6dd' '5027c7'
 '5025ec' '40eb84' '428100' '40c3a3' '426cd7' '40cba2' '40e7aa' '40e928'
 '452920' '45df55' '40b9d9' '40ea32' '40caa5' '501f70' '426916' '40c3e1'
 '402ec6' '43ffe3' '40c44d' '40bdc4' '46c96b' '46c6d9' '4079e4' '5025f2'
 '502807' '46c47e' '4027a5' '50281f' '45df4d' '454536' '40c3c4' '501cfd']


### IPs for which prefetch was triggered in miss_stream config but not in all_pf-miss-train config

In [28]:
difference_set = set(series2) - set(series1)

# Convert back to a Series if needed
difference_series = pd.Series(list(difference_set)).unique()

print(difference_series)

['4098b1' '40bf09' '428145' '4283db' '40bb04' '441710' '45e4ba' '455045'
 '4283f9' '426b4e' '452930' '43fd7d' '40defd' '454670' '45503c' '4024bf'
 '40c413' '40deb1' '45e4b7' '40c502' '502d05' '40e912' '45451b' '4550a8'
 '428456' '40b9c9' '426413' '454512' '44a8dc' '426d22' '40bdf2' '406ca6'
 '455080' '4578de' '45453a' '40e1ce' '40c555' '40e818' '402e77' '40bfa6'
 '402503' '425add' '43fca0' '40be80' '42844f' '4403b9' '44a8c9' '4544f0'
 '454782' '402ed0' '45e4b1' '4403b0' '4282a5' '44046b' '40bd80' '45521c'
 '440468' '40dd26' '426a63' '4626ee' '45526e' '441649' '40e060' '457dd2'
 '440365']


In [29]:
ipcp_isb_pf_df['PC'].value_counts()

PC
45435a    717092
454351    673107
454349    406160
45436e    388215
454372    378676
           ...  
5044dd         1
46c6d9         1
40c37c         1
501f24         1
454f1d         1
Name: count, Length: 285, dtype: int64

In [30]:
ipcp_isb_pf_df_ms['PC'].value_counts()

PC
45435a    124868
454351    119582
454372     82240
45436e     80111
507080     48707
           ...  
40abb7         2
447be1         1
426a63         1
45df60         1
502d05         1
Name: count, Length: 246, dtype: int64

In [31]:
ipcp_isb_pf_df[ipcp_isb_pf_df['PC'] == '45435a']

,PC,Trigger Physical_Address,Trigger Structural_Address,Prefetch Target
0,45435a,805e0c0,400,802a180
1,45435a,805e0c0,400,802e8c0
2,45435a,805e0c0,400,8043540
3,45435a,805e0c0,400,ab50d00
4,45435a,805e0c0,400,820fc00
...,...,...,...,...
5605141,45435a,b0cf0c0,23a0805,80b7980
5605142,45435a,b0cf0c0,23a0805,bf36680
5605220,45435a,be685c0,495404,ae12000
5605223,45435a,bea2bc0,23a0806,80b7980


In [32]:
ipcp_isb_pf_df_ms[ipcp_isb_pf_df_ms['PC'] == '45435a']

,PC,Trigger Physical_Address,Trigger Structural_Address,Prefetch Target
12,45435a,ac969c0,1007,974db40
13,45435a,ac969c0,1007,81b2d00
14,45435a,ac969c0,1007,bbec3c0
15,45435a,ac969c0,1007,802b940
16,45435a,ac969c0,1007,802d100
...,...,...,...,...
1681519,45435a,b0fc000,3811005,889a380
1681555,45435a,80a5c80,2798401,80a8c00
1681633,45435a,b0cf0c0,289d008,beca3c0
1681634,45435a,b0d4fc0,387f000,b1d0700
